In [ ]:
import numpy as np
import pandas as pd
import re
import string
import pandas_import_and_export

# Summer Reading Book Clustering, part 1: Initial cleaning
In this notebook, we will clean the self-reported summer reading data in an attempt to make the titles and authors more comparable and remove unusable entries.

First, import the csv of user-entered books as a pandas dataframe.

In [ ]:
df = pandas_import_and_export.read_csv("bookslogged")  
df[:10]

Clean entries, removing a few things the openlibrary search doesnt like that show up in many entries.

In [ ]:
def clean_title(s): 
    s = s.strip().lower()
    
    #search for some version of "vol. 3" or "v.5" at end 
    #and remove it if it exists
    match = re.search(r",? +v(ol)? *\.? *[1-9]+$",  s)
    if match:
        #print("Removing vol:")
        #print(s)
        s = s[:match.start()].strip()
        #print("-->" + s)
        
    #search for some version of a date at beginning and remove
    #two forms are common in the data: "(1994) title" and "1994 - title"
    #search for those forms separately and specifically  in order to not delete dates in other forms
    #because those tend to be part of the title
    match = re.match(r"\([0-9]{4}\) ",  s)
    if not match: match = re.match(r"[0-9]{4} -",  s)
    if match:
        #print("Removing date at beginning:")
        #print(s)
        s = s[match.end():].strip()
        #print("-->" + s)

    return s

def clean_author(s):
    s = s.strip().lower()
    return s
        
df['title'] = df['title'].apply(clean_title)
df['author'] = df['author'].apply(clean_author)
df[:10]

Delete "bad" rows (no title, or weird short title and no author)

In [ ]:
def isBadTitle(title):
    #defines 'bad" as:
    #a sequence of digits and punctation with lentgh between 0 and 3
    if re.match(
        r"([" + string.digits + string.punctuation + r"]*)$",  
        title
    ): return True
    else: return False

badrows = []
for i, row in df.iterrows():
    if (row.title == "logged a book, but no title"
    or (isBadTitle(row.title) and row.author == "")):
        badrows.append(i)
        
df = df.drop(badrows)
df[:10]

Group together rows whose title and author fields match exactly, adding up times they were read ('sum') and keeping track of number of rows in each group ('count').

In [ ]:
df_grouped = df.groupby(
    ['title', 'author'],
    
    as_index = False
    #this is so that title and author will be made into normal cols rather than
    #weird index cols
).agg(
    {'value' : ['sum', 'count']}
    #sum up the "value" columns for each group.
    #"count" just counts the number of rows in each group (will become "num_readers")
)

#rename to more sensible column names for the sum and count
df_grouped.columns = ['title', 'author', 'num_times_read', 'num_readers']

df_grouped[:10]

In [ ]:
pandas_import_and_export.to_csv(df_grouped, "bookslogged_clean")